In [83]:
from selenium import webdriver
import time
import urllib
import urllib3
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver.common.keys import Keys
from helium import *
from datetime import date
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import openpyxl
import xlsxwriter

def start_writedom():
    try:
        print("Please Wait. Launching Chrome")
        global browser
        options = webdriver.ChromeOptions()
        options.page_load_strategy = 'normal'
        prefs = {"profile.default_content_setting_values.notifications": 2,
                 "credentials_enable_service": False,
                 "profile.password_manager_enabled": False}
        options.add_experimental_option("prefs", prefs)
        # options.add_argument("start-maximized")
        options.add_argument("--disable-extensions")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        browser = webdriver.Chrome(options=options,
                                   executable_path=r'C:\Program Files\Google\Chrome\Application\chromedriver.exe')
        time.sleep(2)
        browser.set_window_size(1225, 960)
        print("Browser launched successfully")
        
    except Exception as e:
        print("This error122: ", e)
        try:
            browser.close()
        except Exception as e:
            print("This error123: ", e)
        start_writedom()
        
def launch():
    # Open the first tab for the markets
    browser.switch_to.window(browser.window_handles[0])
    url = "https://odibets.com/league"
    mrkt_pg = browser.get(url)

    # Open a new window for results
    browser.execute_script("window.open('');")
    # Switch to the new window 
    browser.switch_to.window(browser.window_handles[1])
    rslt_pg = browser.get(url)
    browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[1]/ul[2]/li[2]').click()

    # Open a new window for standings
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[2])
    std_pg = browser.get(url)
    browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[1]/ul[2]/li[3]/button').click()


    
start_writedom()  
time.sleep(4)
launch()




In [154]:
   
def stats_capture():  
    # HDW odds 
    helium.set_driver(browser)

    dct1 = {"Home team":[], "Away team":[],"1":[],"X":[],"2":[], 
            "Time": [], "Date":[], "GG":[],"NG":[], 
            "OV1.5":[], "UN1.5":[], "H standing":[], "A standing":[],
            "H points":[], "A points": [],"H Form points":[],"A Form points":[],"H W":[], "H D": [],
            "H L":[],"A W":[], "A D": [], "A L":[]}


    # This is for the remaining time
    browser.switch_to.window(browser.window_handles[0])
    rm1 = []
    for rema in browser.find_elements_by_class_name("md"):
        rema = int(rema.text.replace(":",""))
        rm1.append(rema)


    # Click the 1X2 button to make sure we are picking the right table
    browser.implicitly_wait(10)
    ele3 = browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/div/button[1]')
    helium.click(ele3)
    time.sleep(1.2)
    browser.implicitly_wait(10)

    page = browser.page_source
    soup = BeautifulSoup(page, 'lxml')
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'event-t'))).text

    table = soup.findAll("div", {"class": "event"})

    home_teams = []
    away_teams = []
    for matchday in table: # For 1X2 odds
        matchday = matchday.text
        matchday = matchday.replace(" 1 ","")
        matchday = matchday.replace("X ", ",")
        matchday1 = matchday.replace("2 ",",")
        for l,d in enumerate(matchday1):
            odds = []
            if d.isdigit():
                matchday2 = matchday1[l:]
                matchday3 = matchday1[:l]
                break

        odds = matchday2.split(",")
        h_odd = odds[0]
        d_odd = odds[1]
        a_odd = odds[2]

        teams = matchday3.strip().split("—")
        h_team = teams[0].strip()
        home_teams.append(h_team)
        a_team = teams[1].strip()
        away_teams.append(a_team)

        dct1["Home team"].append(h_team)
        dct1["Away team"].append(a_team)
        dct1["1"].append(h_odd)
        dct1["X"].append(d_odd)
        dct1["2"].append(a_odd)
        dct1["Time"].append(str(rm1[0] - 2))
        dct1["Date"].append(str(date.today())) # format (year-month-date)


    #Get GG/NG odds
    browser.switch_to.window(browser.window_handles[0])
    browser.implicitly_wait(10)
    ele1 = browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/div/button[2]')
    helium.click(ele1)
    time.sleep(1)
    page1 = browser.page_source
    soup = BeautifulSoup(page1, 'lxml')
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'event-t'))).text
    ggs = soup.findAll("div", {"class": "d-1"})

    for gg in ggs:
        gg = gg.text
        gg = gg.replace("Yes ","")
        gg = gg.replace("No ","")
        ggyes = gg[:-4]
        ggno = gg[-4:]
        dct1["GG"].append(ggyes)
        dct1["NG"].append(ggno)


    # Get over and under 1.5 odds
    browser.implicitly_wait(10)
    ele2 = browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/div/button[3]')
    helium.click(ele2)
    time.sleep(1)
    page2 = browser.page_source
    soup = BeautifulSoup(page2, 'lxml')
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'event-t'))).text
    ovun = soup.findAll("div", {"class": "d-1"})

    for tt in ovun:
        tt = tt.text
        tt = tt.replace("Under ","")
        tt = tt.replace("Over ","")
        ov = tt[-4:]
        un = tt[:-4]
        dct1["OV1.5"].append(ov)
        dct1["UN1.5"].append(un)


    browser.switch_to.window(browser.window_handles[2])
    time.sleep(1.2)
    browser.implicitly_wait(10)
    browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[1]/ul[2]/li[3]/button').click()
    
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'l-league-table-standings'))).text
    time.sleep(1.2)
    standings = pd.read_html(browser.page_source)
    stds = standings[0]

    for one_hteam in home_teams:
        fd = stds[stds['Team'] == one_hteam].index
        pstn = int(fd[0])+1
        dct1["H standing"].append(pstn)

        points = stds.at[int(fd[0]),'Pts']
        dct1["H points"].append(points)

        form = stds.at[int(fd[0]),'Form']
        won, draw, lost = form.count('W'), form.count('D'), form.count('L')
        form_points = (won * 3) + (draw * 1) + (lost * -3)
        dct1["H Form points"].append(form_points)
        dct1["H W"].append(won)
        dct1["H D"].append(draw)
        dct1["H L"].append(lost)

    for one_ateam in away_teams:
        fd1 = stds[stds['Team'] == one_ateam].index
        pstn1 = int(fd1[0])+1
        dct1["A standing"].append(pstn1)

        points1 = stds.at[int(fd1[0]),'Pts']
        dct1["A points"].append(points1)

        form = stds.at[int(fd1[0]),'Form']

        won1, draw1, lost1 = form.count('W'), form.count('D'), form.count('L')
        form_points = (won1 * 3) + (draw1 * 1) + (lost1 * -3)
        dct1["A Form points"].append(form_points)
        dct1["A W"].append(won1)
        dct1["A D"].append(draw1)
        dct1["A L"].append(lost1)
    # print(dct1)
    df_stats = pd.DataFrame.from_dict(dct1)
    # browser.switch_to.window(browser.window_handles[0])
    
    return df_stats


    
def results_capture():
    # Results pd.Dataframe is here
    browser.switch_to.window(browser.window_handles[1])
    df_ff = pd.DataFrame(columns = ['Home team' ,'Scores' ,'Away team', 'Week', 'Season', 'Time'])
    browser.implicitly_wait(10)
    browser.find_element_by_xpath('//*[@id="body"]/div/div[1]/div/div[2]/div[1]/div[1]/ul[2]/li[2]').click()
    time.sleep(1.8)
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'results'))).text

    for g in pd.read_html(browser.page_source):
        dct = {"Home team": [],"Scores": [],"Away team": [],"Week": [], "Season": [], "Time": []}

        t = g[0][0]
        t = t.split("-")
        week = ''
        for wk in t[0]:
            if wk.isdigit():
                week +=wk
        #print(week)
        t = t[1].strip()
        season = int(t[1:6])
        end_time = t.replace(t[:6],"").strip()
        # print(end_time)

        if 'pm' in end_time:
            end_time1 = end_time.split(":")

            if end_time1[0] == '12':
                hrs = str(int(end_time1[0]))
            else:
                hrs = str(int(end_time1[0]) + 12)
            mnts = end_time1[1].replace("pm","")
            end_time2 = int(hrs + mnts)
        else:
            end_time1 = end_time.replace(":","")
            end_time2 = end_time1.replace("am","").strip()
            # print(end_time2)

        # Remove the heading part. Really gave me a hard time figuring it out
        g = g.drop(g.index[0])


        for hm in g[0]:
            #print("Hm",len(hm))
            # hm_lst.append(hm)
            dct["Home team"].append(hm)

        for scr in g[1]:
            #print("Scr",len(scr))
            # hm_lst.append(hm)
            dct["Scores"].append(scr)

        for awy in g[2]:
            # hm_lst.append(hm)
            #print("awy",len(awy))
            dct["Away team"].append(awy)
            dct["Week"].append(week)
            dct["Season"].append(season)
            dct["Time"].append(str(end_time2))

        df_mt = pd.DataFrame.from_dict(dct)
        df_ff = df_ff.append(df_mt, ignore_index = True)
        

    return df_ff



def combine_data():
    browser.switch_to.window(browser.window_handles[0])
    result = pd.merge(df_stats, df_ff, on=["Time","Home team"], how="left")
    pd.set_option('display.max_columns', None)
    return result

def save_to_excel():
    workbook_obj = openpyxl.load_workbook("odidata.xlsx")
    sheet_obj = workbook_obj.active
    for mtc in range(len(result)):
        sheet_obj.append(list(result.iloc[mtc]))
    workbook_obj.save("odidata.xlsx")


In [ ]:
# This is the logical control loop for the scripts

a = []
while True:
    browser.switch_to.window(browser.window_handles[0])
    
    try:
        browser.implicitly_wait(10)
        r1 = browser.find_element_by_class_name("lr").text
    except:
        time.sleep(1)
        continue
    if ":" in r1 and len(a) < 1:
        print("Counting")
        time.sleep(2)
        df_stats = stats_capture()
        a.append('p')
        
    if "LIVE" in r1:
        print("Live")
        time.sleep(2)
    if "END" in r1:
        print("ENDED")
        time.sleep(5)
        df_ff = results_capture()
        time.sleep(2)
        result = combine_data()
        time.sleep(2)
        save_to_excel()
        a.clear()
        time.sleep(2)
        
    time.sleep(1)

In [58]:
# This is the data and all the relevant statistics
df_stats

,Home team,Away team,1,X,2,Time,Date,GG,NG,OV1.5,UN1.5,H standing,A standing,H points,A points,H Form points,A Form points,H W,H D,H L,A W,A D,A L
0,Everton,Manchester Reds,2.61,3.36,2.61,1410,2022-02-26,1.68,2.13,1.28,3.52,12,11,16,16,-1,-5,1,2,2,1,1,3
1,Manchester Blue,London Reds,1.86,3.47,4.21,1410,2022-02-26,1.76,2.01,1.29,3.47,3,16,27,13,1,1,2,1,2,0,4,1
2,West Ham,London Blues,2.84,3.52,2.34,1410,2022-02-26,1.67,2.14,1.27,3.59,5,2,23,35,3,9,3,0,2,4,0,1
3,Aston V,West Brom,2.30,3.12,3.25,1410,2022-02-26,1.70,2.10,1.29,3.43,15,10,15,17,3,-1,3,0,2,1,2,2
4,Leeds,Brighton,2.43,3.21,2.94,1410,2022-02-26,1.69,2.12,1.29,3.47,14,9,15,19,-5,-1,1,1,3,1,2,2
5,Fulham,Liverpool,5.10,4.67,1.53,1410,2022-02-26,1.68,2.14,1.19,4.43,20,1,8,36,-5,9,1,1,3,4,0,1
6,Sheffield U,Newcastle,2.11,3.25,3.55,1410,2022-02-26,1.72,2.07,1.29,3.44,17,7,13,20,-5,9,1,1,3,2,3,0
7,Leicester,Southampton,2.11,3.25,3.55,1410,2022-02-26,1.72,2.07,1.29,3.44,4,19,24,10,7,-3,3,1,1,2,0,3
8,Burnley,Tottenham,2.39,3.18,3.01,1410,2022-02-26,1.69,2.12,1.29,3.47,18,13,11,15,-5,5,1,1,3,2,2,1
9,Wolves,Palace,2.00,3.35,3.78,1410,2022-02-26,1.74,2.05,1.29,3.45,6,8,22,20,3,5,3,0,2,2,2,1


In [59]:
# This is the corresponding results
df_ff

,Home team,Scores,Away team,Week,Season,Time
0,Wolves,20,Palace,18,29544,1410
1,Burnley,10,Tottenham,18,29544,1410
2,Leicester,20,Southampton,18,29544,1410
3,Sheffield U,12,Newcastle,18,29544,1410
4,Fulham,12,Liverpool,18,29544,1410
...,...,...,...,...,...,...
375,Leicester,23,Leeds,19,29543,1256
376,Liverpool,11,Wolves,19,29543,1256
377,Brighton,11,West Ham,19,29543,1256
378,Manchester Reds,11,Manchester Blue,19,29543,1256


In [60]:
# combined data now
result

,Home team,Away team_x,1,X,2,Time,Date,GG,NG,OV1.5,UN1.5,H standing,A standing,H points,A points,H Form points,A Form points,H W,H D,H L,A W,A D,A L,Scores,Away team_y,Week,Season
0,Everton,Manchester Reds,2.61,3.36,2.61,1410,2022-02-26,1.68,2.13,1.28,3.52,12,11,16,16,-1,-5,1,2,2,1,1,3,12,Manchester Reds,18,29544
1,Manchester Blue,London Reds,1.86,3.47,4.21,1410,2022-02-26,1.76,2.01,1.29,3.47,3,16,27,13,1,1,2,1,2,0,4,1,41,London Reds,18,29544
2,West Ham,London Blues,2.84,3.52,2.34,1410,2022-02-26,1.67,2.14,1.27,3.59,5,2,23,35,3,9,3,0,2,4,0,1,11,London Blues,18,29544
3,Aston V,West Brom,2.30,3.12,3.25,1410,2022-02-26,1.70,2.10,1.29,3.43,15,10,15,17,3,-1,3,0,2,1,2,2,02,West Brom,18,29544
4,Leeds,Brighton,2.43,3.21,2.94,1410,2022-02-26,1.69,2.12,1.29,3.47,14,9,15,19,-5,-1,1,1,3,1,2,2,21,Brighton,18,29544
5,Fulham,Liverpool,5.10,4.67,1.53,1410,2022-02-26,1.68,2.14,1.19,4.43,20,1,8,36,-5,9,1,1,3,4,0,1,12,Liverpool,18,29544
6,Sheffield U,Newcastle,2.11,3.25,3.55,1410,2022-02-26,1.72,2.07,1.29,3.44,17,7,13,20,-5,9,1,1,3,2,3,0,12,Newcastle,18,29544
7,Leicester,Southampton,2.11,3.25,3.55,1410,2022-02-26,1.72,2.07,1.29,3.44,4,19,24,10,7,-3,3,1,1,2,0,3,20,Southampton,18,29544
8,Burnley,Tottenham,2.39,3.18,3.01,1410,2022-02-26,1.69,2.12,1.29,3.47,18,13,11,15,-5,5,1,1,3,2,2,1,10,Tottenham,18,29544
9,Wolves,Palace,2.00,3.35,3.78,1410,2022-02-26,1.74,2.05,1.29,3.45,6,8,22,20,3,5,3,0,2,2,2,1,20,Palace,18,29544
